# Debug Drill: The Forgotten Order

**Scenario:**
Your support chatbot has been handling long conversations, but users are complaining.

"I told it my order number at the start, but now it's asking me again!" a frustrated user reports.

The agent's memory is overflowing, and critical information is being lost.

**Your Task:**
1. Identify why the agent forgets the order number
2. Implement a fix using entity memory
3. Verify critical info is preserved
4. Write a 3-bullet postmortem

---

In [ ]:
import re
from dataclasses import dataclass
from datetime import datetime
from typing import List, Dict, Optional

In [ ]:
@dataclass
class Message:
    role: str
    content: str
    tokens: int = 0
    
    def __post_init__(self):
        self.tokens = len(self.content) // 4

In [ ]:
# ===== COLLEAGUE'S CODE (BUG: No entity preservation) =====

class BrokenConversationMemory:
    """Memory that loses important info when context overflows."""
    
    def __init__(self, max_tokens: int = 100):
        self.messages: List[Message] = []
        self.max_tokens = max_tokens
    
    def add(self, role: str, content: str):
        msg = Message(role=role, content=content)
        self.messages.append(msg)
        self._enforce_limit()
    
    def _enforce_limit(self):
        """BUG: Just removes oldest messages, no matter what they contain!"""
        total = sum(m.tokens for m in self.messages)
        while total > self.max_tokens and len(self.messages) > 1:
            removed = self.messages.pop(0)
            total -= removed.tokens
            # BUG: Doesn't check if removed message contains critical info!
    
    def get_context(self) -> str:
        return "\n".join([f"{m.role}: {m.content}" for m in self.messages])
    
    def get_order_id(self) -> Optional[str]:
        """Try to find order ID in remaining context."""
        context = self.get_context()
        match = re.search(r'ORD-\d+', context)
        return match.group() if match else None

In [ ]:
# Simulate a long conversation
memory = BrokenConversationMemory(max_tokens=100)  # Small limit for demo

conversation = [
    ("user", "Hi, I need help with order ORD-12345"),
    ("assistant", "I'd be happy to help with order ORD-12345! What's the issue?"),
    ("user", "The item arrived damaged"),
    ("assistant", "I'm sorry to hear that. I can arrange a replacement."),
    ("user", "How long will that take?"),
    ("assistant", "Replacements typically arrive in 3-5 business days."),
    ("user", "Can you also check if I have any coupons?"),
    ("assistant", "Let me check your account for available coupons."),
    ("user", "Great, and when will my replacement ship?"),
]

print("=== Simulating Conversation ===")
for role, content in conversation:
    memory.add(role, content)
    order_id = memory.get_order_id()
    print(f"[{len(memory.messages)} msgs] Order ID in context: {order_id}")

print(f"\n❌ PROBLEM: Order ID is now {memory.get_order_id()}!")
print("   The agent will have to ask the user again.")

---

## Your Investigation

### Step 1: Understand why entities are lost

In [ ]:
print("=== Problem Analysis ===")
print()
print("The broken memory system:")
print("  1. Only uses a sliding window (oldest first)")
print("  2. Doesn't track important entities")
print("  3. When context fills up, critical info is lost")
print()
print("Current context:")
print(memory.get_context())

### Step 2: TODO - Implement entity-aware memory

In [ ]:
# TODO: Create memory that preserves critical entities

# Uncomment and complete:

# class EntityAwareMemory:
#     """Memory that preserves critical entities."""
#     
#     def __init__(self, max_tokens: int = 100):
#         self.messages: List[Message] = []
#         self.max_tokens = max_tokens
#         self.entities = {}  # Track extracted entities
#     
#     def add(self, role: str, content: str):
#         msg = Message(role=role, content=content)
#         self.messages.append(msg)
#         self._extract_entities(content)
#         self._enforce_limit()
#     
#     def _extract_entities(self, text: str):
#         """Extract and save important entities."""
#         # Extract order IDs
#         orders = re.findall(r'ORD-\d+', text)
#         for order in orders:
#             self.entities['order_id'] = order.upper()
#         
#         # Extract emails
#         emails = re.findall(r'[\w.-]+@[\w.-]+\.\w+', text)
#         for email in emails:
#             self.entities['email'] = email
#     
#     def _enforce_limit(self):
#         """Remove old messages but preserve entities."""
#         total = sum(m.tokens for m in self.messages)
#         while total > self.max_tokens and len(self.messages) > 1:
#             removed = self.messages.pop(0)
#             total -= removed.tokens
#     
#     def get_context(self) -> str:
#         """Get context with preserved entities."""
#         # Add entity summary at the start
#         entity_summary = ""
#         if self.entities:
#             entity_summary = "Key facts: " + ", ".join([f"{k}={v}" for k,v in self.entities.items()]) + "\n\n"
#         
#         messages = "\n".join([f"{m.role}: {m.content}" for m in self.messages])
#         return entity_summary + messages
#     
#     def get_order_id(self) -> Optional[str]:
#         """Get order ID from entity store."""
#         return self.entities.get('order_id')
# 
# print("✓ EntityAwareMemory defined")

In [ ]:
# TODO: Test the fixed memory

# Uncomment:

# fixed_memory = EntityAwareMemory(max_tokens=100)
# 
# print("=== Testing Fixed Memory ===")
# for role, content in conversation:
#     fixed_memory.add(role, content)
#     order_id = fixed_memory.get_order_id()
#     print(f"[{len(fixed_memory.messages)} msgs] Order ID: {order_id}")
# 
# print(f"\n✓ Order ID preserved: {fixed_memory.get_order_id()}")
# print(f"\nContext with entities:")
# print(fixed_memory.get_context()[:200] + "...")

In [ ]:
# ============================================
# SELF-CHECK
# ============================================

# Uncomment:

# assert fixed_memory.get_order_id() == "ORD-12345", "Should preserve order ID"
# assert "ORD-12345" in fixed_memory.get_context(), "Context should include order ID"
# 
# print("✓ Memory management fixed!")
# print(f"✓ Order ID preserved across {len(conversation)} messages")

### Step 3: Write your postmortem

In [ ]:
postmortem = """
## Postmortem: The Forgotten Order

### What happened:
- (Your answer: What user complaint indicated the problem?)

### Root cause:
- (Your answer: Why did the memory system lose the order ID?)

### How to prevent:
- (Your answer: What memory strategy preserves critical entities?)

"""

print(postmortem)

---

## ✅ Drill Complete!

**Key lessons:**

1. **Sliding window alone isn't enough.** Critical info at the start gets lost.

2. **Extract entities as they appear.** Store them separately from the conversation.

3. **Prepend entities to context.** Ensures they're always available.

4. **Test with long conversations.** Short tests won't reveal overflow bugs.

---

## Memory Strategy Guide

| Strategy | Pros | Cons |
|----------|------|------|
| Sliding window | Simple | Loses early context |
| Entity extraction | Preserves facts | Misses nuance |
| Summarization | Preserves gist | Expensive |
| Hybrid | Best of both | More complex |